In [1]:
from __future__ import division, print_function, absolute_import
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import numpy
import PIL
from PIL import Image
np.random.seed(1337)  # for reproducibility

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [57]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10

# Network Parameters
num_hidden_1 = 64 # 1st layer num features
num_hidden_2 = 16 # 2nd layer num features (the latent dim)
num_input = 43 
num_classes = 5

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_2, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [58]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [13]:
from numpy import genfromtxt
my_data = genfromtxt('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/Cgamma_VHc.csv', delimiter=',')
my_data1 = genfromtxt('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/Cgamma_VVc.csv', delimiter=',')

In [14]:
data = my_data[1:,2:]
data1 = my_data1[1:,2:]
print(data.shape)
print(data.shape)

(291, 43)
(291, 43)


In [15]:
print(data.max())

0.162946105003357


In [16]:
data = np.array(data)
data1 = np.array(data1)
print(data.shape)
print(data1.shape)

(291, 43)
(291, 43)


In [8]:
import pandas
colnames = ['Latitude', 'Longitude', 'CType', 'Remarks', 'Index']
data1 = pandas.read_csv('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/CropClean.csv', names=colnames)

In [9]:
ctype = data1.CType.tolist()
print(ctype)

['CType', 'PD', 'PD', 'PD', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'JW', 'SC', 'SC', 'SC', 'PP', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'BA', 'BA', 'SC', 'SC', 'SC', 'JW', 'PD', 'SC', 'BA', 'SC', 'BA', 'PD', 'PD', 'SC', 'PP', 'PD', 'PD', 'PD', 'PD', 'PD', 'BA', 'SC', 'SC', 'BA', 'BA', 'JW', 'JW', 'SC', 'PD', 'BA', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'PD', 'BA', 'SC', 'PD', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'JW', 'PD', 'PD', 'SC', 'JW', 'PD', 'PD', 'BA', 'BA', 'SC', 'SC', 'SC', 'BA', 'PP', 'PP', 'BA', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'PP', 'SC', 'SC', 'SC', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'JW', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'BA', 'BA', 'PD', 'PD', 'SC', 'PD', 'SC', 'PD', 'BA', 'JW', 'PD', 'SC', 'JW', 'SC', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'PD', 'SC', 'SC', 'PD', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'JW', 'PD', 'SC', 'SC', 'JW', 

In [10]:
p = np.array(ctype)
print(p.shape)

(292,)


In [11]:
lbl1 = ctype
label = []
for i in range(1,292):
    if(lbl1[i] == "PD"):
        label += [0]
    if(lbl1[i] == "JW"):
        label += [1]
    if(lbl1[i] == "PP"):
        label += [2]
    if(lbl1[i] == "SC"):
        label += [3]
    if(lbl1[i] == "BA"):
        label += [4]
print(label)

[0, 0, 0, 3, 1, 3, 3, 3, 3, 1, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 1, 0, 3, 4, 3, 4, 0, 0, 3, 2, 0, 0, 0, 0, 0, 4, 3, 3, 4, 4, 1, 1, 3, 0, 4, 1, 3, 3, 3, 3, 3, 0, 0, 4, 3, 0, 0, 3, 3, 3, 3, 3, 1, 0, 0, 3, 1, 0, 0, 4, 4, 3, 3, 3, 4, 2, 2, 4, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 1, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 0, 0, 3, 0, 3, 0, 4, 1, 0, 3, 1, 3, 3, 1, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 3, 1, 3, 3, 3, 3, 3, 3, 0, 1, 0, 3, 3, 1, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3, 3, 3, 0, 0, 0, 0, 1, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 4, 4, 3, 0, 4, 0, 3, 3, 3, 3, 0, 3, 0, 3, 2, 3, 0, 3, 1, 4, 3, 0, 0, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 4, 1, 0, 0, 2, 3, 3, 3, 0, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 0, 4, 4, 4, 0, 0, 0, 0, 3, 3, 3, 3, 4, 3, 3, 4, 4, 3, 3, 1, 3, 4, 1, 3, 3, 3, 3, 4, 0, 0]


In [12]:
label = np.array(label)
print(label.shape)

(291,)


In [17]:
data2 = data1/data
print(data2.shape)

(291, 43)


In [19]:
print(data2)
print(data2.max())
data2 = data2/data2.max()
print(data2.max())

[[3.97145887 3.66846041 4.4551523  ... 2.40375025 2.46242098 4.58025963]
 [3.09201878 3.02861886 4.23078402 ... 2.65462117 5.10168325 5.85614705]
 [3.12849236 4.57329223 2.23865943 ... 1.3642738  2.67007095 5.48129715]
 ...
 [2.73395896 2.86940239 3.29195385 ... 3.3320972  3.04228477 8.16708078]
 [4.58948135 6.94370217 7.30208959 ... 9.11640627 8.9000907  3.21773101]
 [7.18431668 4.71121624 5.16515067 ... 3.76544599 3.1413722  3.63696308]]
29.889381818954696
1.0


In [47]:
# 80% train 20% test (NDVIclean)
X_train = data2[0:232,:]
X_test = data2[232:,:]
y_train = label[0:232]
y_test = label[232:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(232, 43)
(59, 43)
(232,)
(59,)


In [48]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(232, 5)
(59, 5)


In [49]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 17
num_batch = 13

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.73126256
Epoch: 2 cost = 0.67409638
Epoch: 3 cost = 0.61882947
Epoch: 4 cost = 0.56734289
Epoch: 5 cost = 0.52131312
Epoch: 6 cost = 0.48185669
Epoch: 7 cost = 0.44943336
Epoch: 8 cost = 0.42391447
Epoch: 9 cost = 0.40470169
Epoch: 10 cost = 0.39086762
Epoch: 11 cost = 0.38132191
Epoch: 12 cost = 0.37497895
Epoch: 13 cost = 0.37088506
Epoch: 14 cost = 0.36828562
Epoch: 15 cost = 0.36663305
Epoch: 16 cost = 0.36555716
Epoch: 17 cost = 0.36481872
Epoch: 18 cost = 0.36426513
Epoch: 19 cost = 0.36379680
Epoch: 20 cost = 0.36334606
Epoch: 21 cost = 0.36286514
Epoch: 22 cost = 0.36231930
Epoch: 23 cost = 0.36168236
Epoch: 24 cost = 0.36093369
Epoch: 25 cost = 0.36005690
Epoch: 26 cost = 0.35904092
Epoch: 27 cost = 0.35788334
Epoch: 28 cost = 0.35659403
Epoch: 29 cost = 0.35519325
Epoch: 30 cost = 0.35370040
Epoch: 31 cost = 0.35211635
Epoch: 32 cost = 0.35041510
Epoch: 33 cost = 0.34855338
Epoch: 34 cost = 0.34648708
Epoch: 35 cost = 0.34418187
Epoch: 36 cost = 0.34161491
E

Epoch: 298 cost = 0.00234315
Epoch: 299 cost = 0.00135177
Epoch: 300 cost = 0.00100655
Epoch: 301 cost = 0.00085533
Epoch: 302 cost = 0.00081669
Epoch: 303 cost = 0.00078453
Epoch: 304 cost = 0.00079903
Epoch: 305 cost = 0.00080180
Epoch: 306 cost = 0.00078324
Epoch: 307 cost = 0.00076601
Epoch: 308 cost = 0.00077860
Epoch: 309 cost = 0.00072676
Epoch: 310 cost = 0.00069530
Epoch: 311 cost = 0.00075075
Epoch: 312 cost = 0.00074479
Epoch: 313 cost = 0.00069071
Epoch: 314 cost = 0.00070657
Epoch: 315 cost = 0.00076885
Epoch: 316 cost = 0.00072322
Epoch: 317 cost = 0.00069209
Epoch: 318 cost = 0.00071247
Epoch: 319 cost = 0.00065728
Epoch: 320 cost = 0.00062832
Epoch: 321 cost = 0.00066213
Epoch: 322 cost = 0.00071443
Epoch: 323 cost = 0.00073130
Epoch: 324 cost = 0.00065960
Epoch: 325 cost = 0.00068634
Epoch: 326 cost = 0.00069586
Epoch: 327 cost = 0.00064661
Epoch: 328 cost = 0.00062016
Epoch: 329 cost = 0.00063000
Epoch: 330 cost = 0.00068853
Epoch: 331 cost = 0.00070571
Epoch: 332 cos

Epoch: 693 cost = 0.00022741
Epoch: 694 cost = 0.00021753
Epoch: 695 cost = 0.00023093
Epoch: 696 cost = 0.00025624
Epoch: 697 cost = 0.00022088
Epoch: 698 cost = 0.00024021
Epoch: 699 cost = 0.00028175
Epoch: 700 cost = 0.00029022
Epoch: 701 cost = 0.00026699
Epoch: 702 cost = 0.00023961
Epoch: 703 cost = 0.00020903
Epoch: 704 cost = 0.00021908
Epoch: 705 cost = 0.00022529
Epoch: 706 cost = 0.00021999
Epoch: 707 cost = 0.00023724
Epoch: 708 cost = 0.00027833
Epoch: 709 cost = 0.00031986
Epoch: 710 cost = 0.00029398
Epoch: 711 cost = 0.00036406
Epoch: 712 cost = 0.00024707
Epoch: 713 cost = 0.00024106
Epoch: 714 cost = 0.00019809
Epoch: 715 cost = 0.00020393
Epoch: 716 cost = 0.00019819
Epoch: 717 cost = 0.00018517
Epoch: 718 cost = 0.00022340
Epoch: 719 cost = 0.00024766
Epoch: 720 cost = 0.00032063
Epoch: 721 cost = 0.00031056
Epoch: 722 cost = 0.00027752
Epoch: 723 cost = 0.00027028
Epoch: 724 cost = 0.00021759
Epoch: 725 cost = 0.00020336
Epoch: 726 cost = 0.00019748
Epoch: 727 cos

In [61]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [51]:
print(y_test.shape)

(59, 5)


In [52]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.55932206


In [53]:
# 70% train 30% test (NDVIclean)
# Train - 204
# Test - 87
X_train = data2[0:204,:]
X_test = data2[204:,:]
y_train = label[0:204]
y_test = label[204:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(204, 43)
(87, 43)
(204,)
(87,)


In [54]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(204, 5)
(87, 5)


In [59]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 17
num_batch = 12

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.65443387
Epoch: 2 cost = 0.59048726
Epoch: 3 cost = 0.53950836
Epoch: 4 cost = 0.50221662
Epoch: 5 cost = 0.47686674
Epoch: 6 cost = 0.46050551
Epoch: 7 cost = 0.45027556
Epoch: 8 cost = 0.44397475
Epoch: 9 cost = 0.44007291
Epoch: 10 cost = 0.43756121
Epoch: 11 cost = 0.43580119
Epoch: 12 cost = 0.43440857
Epoch: 13 cost = 0.43316438
Epoch: 14 cost = 0.43195051
Epoch: 15 cost = 0.43070477
Epoch: 16 cost = 0.42939325
Epoch: 17 cost = 0.42799596
Epoch: 18 cost = 0.42650020
Epoch: 19 cost = 0.42489813
Epoch: 20 cost = 0.42318501
Epoch: 21 cost = 0.42135922
Epoch: 22 cost = 0.41942034
Epoch: 23 cost = 0.41736855
Epoch: 24 cost = 0.41520341
Epoch: 25 cost = 0.41292316
Epoch: 26 cost = 0.41052391
Epoch: 27 cost = 0.40799952
Epoch: 28 cost = 0.40534163
Epoch: 29 cost = 0.40253985
Epoch: 30 cost = 0.39958221
Epoch: 31 cost = 0.39645567
Epoch: 32 cost = 0.39314701
Epoch: 33 cost = 0.38964373
Epoch: 34 cost = 0.38593507
Epoch: 35 cost = 0.38201408
Epoch: 36 cost = 0.37787942
E

Epoch: 377 cost = 0.00056755
Epoch: 378 cost = 0.00061272
Epoch: 379 cost = 0.00057520
Epoch: 380 cost = 0.00055569
Epoch: 381 cost = 0.00066710
Epoch: 382 cost = 0.00066414
Epoch: 383 cost = 0.00059706
Epoch: 384 cost = 0.00055245
Epoch: 385 cost = 0.00056072
Epoch: 386 cost = 0.00055298
Epoch: 387 cost = 0.00050819
Epoch: 388 cost = 0.00057187
Epoch: 389 cost = 0.00061236
Epoch: 390 cost = 0.00057872
Epoch: 391 cost = 0.00057208
Epoch: 392 cost = 0.00053687
Epoch: 393 cost = 0.00069398
Epoch: 394 cost = 0.00073974
Epoch: 395 cost = 0.00059940
Epoch: 396 cost = 0.00056140
Epoch: 397 cost = 0.00061249
Epoch: 398 cost = 0.00050345
Epoch: 399 cost = 0.00054208
Epoch: 400 cost = 0.00052158
Epoch: 401 cost = 0.00050575
Epoch: 402 cost = 0.00049440
Epoch: 403 cost = 0.00053585
Epoch: 404 cost = 0.00071085
Epoch: 405 cost = 0.00077890
Epoch: 406 cost = 0.00070543
Epoch: 407 cost = 0.00052842
Epoch: 408 cost = 0.00054365
Epoch: 409 cost = 0.00050145
Epoch: 410 cost = 0.00049586
Epoch: 411 cos

Epoch: 768 cost = 0.00031046
Epoch: 769 cost = 0.00034790
Epoch: 770 cost = 0.00032215
Epoch: 771 cost = 0.00029204
Epoch: 772 cost = 0.00033184
Epoch: 773 cost = 0.00033171
Epoch: 774 cost = 0.00035320
Epoch: 775 cost = 0.00038714
Epoch: 776 cost = 0.00034678
Epoch: 777 cost = 0.00032313
Epoch: 778 cost = 0.00030840
Epoch: 779 cost = 0.00034655
Epoch: 780 cost = 0.00030102
Epoch: 781 cost = 0.00032889
Epoch: 782 cost = 0.00028840
Epoch: 783 cost = 0.00028263
Epoch: 784 cost = 0.00035165
Epoch: 785 cost = 0.00033960
Epoch: 786 cost = 0.00031925
Epoch: 787 cost = 0.00039007
Epoch: 788 cost = 0.00039309
Epoch: 789 cost = 0.00033982
Epoch: 790 cost = 0.00027316
Epoch: 791 cost = 0.00026630
Epoch: 792 cost = 0.00029663
Epoch: 793 cost = 0.00030176
Epoch: 794 cost = 0.00030394
Epoch: 795 cost = 0.00035726
Epoch: 796 cost = 0.00039050
Epoch: 797 cost = 0.00034120
Epoch: 798 cost = 0.00035348
Epoch: 799 cost = 0.00031288
Epoch: 800 cost = 0.00031790
Epoch: 801 cost = 0.00027433
Epoch: 802 cos

In [62]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.54022986
